# Функции потерь и оптимизация

In [1]:
from sklearn.datasets import load_iris
import pandas as pd
import numpy as np

iris = load_iris()

In [2]:
iris.target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [3]:
type(iris)

sklearn.utils.Bunch

Конвертируем наш датасет в pandas DataFrame

In [4]:
df = pd.DataFrame(data=np.c_[iris['data'], iris['target']], columns=iris.feature_names + ['Species'])
df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),Species
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2.0
146,6.3,2.5,5.0,1.9,2.0
147,6.5,3.0,5.2,2.0,2.0
148,6.2,3.4,5.4,2.3,2.0


Оставляем только 2 класса: Iris Versicolor, Iris Virginica и разбиваем данные на тестовую и обучающую выборки

In [5]:
from sklearn.model_selection import train_test_split

df_2_fetures = df[df['Species'] != 0].reset_index(drop=True)

# 80%/20%
TRAIN_X, TEST_X, TRAIN_Y, TEST_Y = train_test_split(df_2_fetures.iloc[:, :-1], df_2_fetures["Species"].replace(1, 0).replace(2, 1).values.astype('int64'), test_size = 0.2, random_state = 0)

In [6]:
TEST_X

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
26,6.8,2.8,4.8,1.4
86,6.3,3.4,5.6,2.4
2,6.9,3.1,4.9,1.5
55,7.6,3.0,6.6,2.1
75,7.2,3.2,6.0,1.8
93,6.8,3.2,5.9,2.3
16,5.6,3.0,4.5,1.5
73,6.3,2.7,4.9,1.8
54,6.5,3.0,5.8,2.2
95,6.7,3.0,5.2,2.3


In [7]:
TEST_Y

array([0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0])

Класс отвечающий за LogisticRegression с реализацией трех методов - градиентного спуска (gd), метода скользящего среднего (RMSProp), ускоренный по Нестерову метод адаптивной оценки моментов (Nadam). В качестве метрик выбран precision и score.

In [8]:
import time 
import math

class LogReg:
    def __init__(self, lr=0.01, eps=pow(10, -6), func="gd", rho=0.9, b1=0.9, b2=0.999, epochs=None):
        self.lr = lr
        self.eps = eps
        self.W = []
        self.points = []
        self.total_time = 0
        self.rho = rho
        self.b1 = b1
        self.b2 = b2
        self.t = 1
        self.epochs = epochs
        self.func_name = func

        if func == "gd":
            self.func = self.gd
        elif func == "rmsprop":
            self.func = self.rmsprop
        elif func == "nadam":
            self.func = self.nadam

    def fit(self, X, Y):
        self.X = X
        self.Y = Y
        self.W = np.zeros(self.X.shape[1])
        self.cach = np.zeros(self.X.shape[1])
        self.m = np.zeros(self.X.shape[1])
        self.v = np.zeros(self.X.shape[1])
        start = time.time()
        self.steps = 0

        self.points.append(self.cost_function(self.X, self.Y, self.W))

        while True:
            self.func()
            self.steps += 1

            if abs(self.points[-1] - self.points[-2]) < self.eps:
                break
              
            if math.isnan(self.points[-1]):
              print("Last cost value is nan")
              break

            if self.epochs and len(self.points) >= self.epochs:
              print(self.points)
              break

        print("Steps: ", self.steps)
        self.total_time = time.time() - start

    def gd(self):
        self.W -= self.lr * self.gradient(self.X, self.Y, self.W)
        self.points.append(self.cost_function(self.X, self.Y, self.W))

    def rmsprop(self):
        for i, gradient in enumerate(self.gradient(self.X, self.Y, self.W)):
            self.cach[i] = self.rho * self.cach[i] + (1 - self.rho) * (gradient ** 2)
            self.W[i] -= (self.lr / np.sqrt(self.cach[i] + 1e-6)) * gradient
        self.points.append(self.cost_function(self.X, self.Y, self.W))

    def nadam(self):
        for i, gradient in enumerate(self.gradient(self.X, self.Y, self.W)):
            self.m[i] = self.b1 * self.m[i] + (1 - self.b1) * gradient
            self.v[i] = self.b2 * self.v[i] + (1 - self.b2) * gradient ** 2
            m_cor = self.m[i] / (1 - self.b1 ** self.t)
            v_cor = self.v[i] / (1 - self.b2 ** self.t)

            self.W[i] -= (self.lr / (np.sqrt(v_cor) + 1e-8) * (
                        self.b1 * m_cor + (1 - self.b1) * gradient / (1 - self.b1 ** self.t)))

        self.t += 1
        self.points.append(self.cost_function(self.X, self.Y, self.W))

    # Предположим что TP это 0 класс, а TN это 1 класс, посчитаем метрику и найдем среднее
    def precision(self, Y, Ypred):
        tp, fp = 0, 0
        tn, fn = 0, 0

        for i in range(len(Y)):
            if Ypred[i] == Y[i] == 0:
                tp += 1
            elif Ypred[i] == 0 != Y[i]:
                fp += 1
            elif Ypred[i] == Y[i] == 1:
                tn += 1
            elif Ypred[i] == 1 != Y[i]:
                fn += 1

        return (tp / (tp + fp) + tn / (tn + fn)) / 2

    def score(self, Y, Ypred):
        pos_value = 0

        for i in range(len(Y)):
            if Ypred[i] == Y[i]:
                pos_value += 1

        return pos_value / len(Y)
    
    def get_metric(self, Y, Ypred):
      return [self.func_name, self.precision(Y, Ypred), self.score(Y, Ypred), self.steps, self.total_time]

    def predict(self, X):
        Z = self.sigmoid(X.dot(self.W))
        predicted = np.where(Z > 0.5, 1, 0)
        return predicted

    def cost_function(self, X, Y, W):
        n = X.shape[0]
        cost = -(1 / n) * np.sum(
            Y * np.log(self.sigmoid(np.dot(X, W))) + (1 - Y) * np.log(
                1 - self.sigmoid(np.dot(X, W))))
        return cost

    def gradient(self, X, Y, W):
        n = X.shape[0]
        return (1 / n) * np.dot(X.T, self.sigmoid(np.dot(X, W)) - Y)

    def sigmoid(self, X):
        return 1 / (1 + np.exp(-X))


Создадим DataFrame для метрики

In [9]:
metrics = pd.DataFrame(columns=['method', 'precision (metric)', 'score (metric)', 'steps' , 'time'])

Протестируем наши методы

## Градиентный спуск

In [10]:
gd_model = LogReg()
gd_model.fit(TRAIN_X, TRAIN_Y)  

Steps:  32965


In [11]:
gd_predicted = gd_model.predict(TEST_X)
print(gd_predicted)
print(TEST_Y)

[0 1 0 1 1 1 0 1 1 1 1 1 1 0 0 0 1 0 1 0]
[0 1 0 1 1 1 0 1 1 1 1 1 1 0 0 0 0 0 0 0]


In [12]:
gd_model.precision(TEST_Y, gd_predicted)

0.9166666666666667

In [13]:
gd_model.score(TEST_Y, gd_predicted)

0.9

In [14]:
gd_model.total_time

12.996808052062988

In [15]:
metrics.loc[len(metrics)] = gd_model.get_metric(TEST_Y, gd_predicted)

## RMSProp

In [16]:
rmsprop_model = LogReg(func="rmsprop")
rmsprop_model.fit(TRAIN_X, TRAIN_Y)  

Steps:  4633


In [17]:
rmsprop_predicted = rmsprop_model.predict(TEST_X)
print(rmsprop_predicted)
print(TEST_Y)

[0 1 0 1 1 1 0 1 1 1 1 1 1 0 0 0 1 0 1 0]
[0 1 0 1 1 1 0 1 1 1 1 1 1 0 0 0 0 0 0 0]


In [18]:
rmsprop_model.precision(TEST_Y, rmsprop_predicted)

0.9166666666666667

In [19]:
rmsprop_model.score(TEST_Y, rmsprop_predicted)

0.9

In [20]:
rmsprop_model.total_time

1.2350904941558838

In [21]:
metrics.loc[len(metrics)] = rmsprop_model.get_metric(TEST_Y, rmsprop_predicted)

## Nadam

In [22]:
nadam_model = LogReg(func="nadam")
nadam_model.fit(TRAIN_X, TRAIN_Y)  

Steps:  6711


In [23]:
nadam_predicted = nadam_model.predict(TEST_X)
print(nadam_predicted)
print(TEST_Y)

[0 1 0 1 1 1 0 1 1 1 1 1 1 0 0 0 1 0 1 0]
[0 1 0 1 1 1 0 1 1 1 1 1 1 0 0 0 0 0 0 0]


In [24]:
nadam_model.precision(TEST_Y, nadam_predicted)

0.9166666666666667

In [25]:
nadam_model.score(TEST_Y, nadam_predicted)

0.9

In [26]:
nadam_model.total_time

1.962416410446167

In [27]:
metrics.loc[len(metrics)] = nadam_model.get_metric(TEST_Y, nadam_predicted)

## Метрика

In [28]:
metrics

,method,precision (metric),score (metric),steps,time
0,gd,0.916667,0.9,32965,12.996808
1,rmsprop,0.916667,0.9,4633,1.235090
2,nadam,0.916667,0.9,6711,1.962416


## Выводы

По выводу метрики видно что самый быстрый метод это RMSProp, второе место занимает Nadam, а самый медленный метод (более чем в 6 раз медленее чем Nadam) - Градиентный спуск.

В ходе практического задания были изучены 3 подхода к обучению логистической регресии. Проводил эксперименты с метриками и улучшением качества обучения. Ловил баги с зависанием на одном месте из-за маленького learning_rate и правил это.